# Problema

Predecir el coste del seguro

## Instrucciones

 Utilizar el dataset (insurance.csv) para entrenar un modelo de regresión capaz de predecir el valor del seguro en función de las características del cliente. Realizar limpieza, preprocesado modelado y testeo del modelo aportando conclusiones de todos estos pasos.

# El set de datos

* age: age of primary beneficiary

* sex: insurance contractor gender, female, male

* bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

* children: Number of children covered by health insurance / Number of dependents

* smoker: Smoking

* region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

* charges: Individual medical costs billed by health insurance



In [1]:
# imports
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [2]:
ruta = "/content/insurance.csv"
df = pd.read_csv(ruta)
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [3]:
print(df.shape)
df.head()

(1338, 7)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
import pandas as pd

# Utiliza el siguiente código para encontrar los valores nulos en el DataFrame
nulos = df.isnull().sum()
print(nulos)


age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [6]:
# Obtener la distribución de valores para cada variable
for columna in df.columns:
    reparto_valores = df[columna].value_counts()

    # Imprimir el reparto de valores para la columna actual
    print(f"Distribución de valores para la columna '{columna}':")
    print(reparto_valores)
    print()


Distribución de valores para la columna 'age':
age
18    69
19    68
50    29
51    29
47    29
46    29
45    29
20    29
48    29
52    29
22    28
49    28
54    28
53    28
21    28
26    28
24    28
25    28
28    28
27    28
23    28
43    27
29    27
30    27
41    27
42    27
44    27
31    27
40    27
32    26
33    26
56    26
34    26
55    26
57    26
37    25
59    25
58    25
36    25
38    25
35    25
39    25
61    23
60    23
63    23
62    23
64    22
Name: count, dtype: int64

Distribución de valores para la columna 'sex':
sex
male      676
female    662
Name: count, dtype: int64

Distribución de valores para la columna 'bmi':
bmi
32.300    13
28.310     9
30.495     8
30.875     8
31.350     8
          ..
46.200     1
23.800     1
44.770     1
32.120     1
30.970     1
Name: count, Length: 548, dtype: int64

Distribución de valores para la columna 'children':
children
0    574
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64

Distribución de v

Analizando el dataset, vemos que no existen valores nulos, ni ninguna variable con outliers o valores anómalos, porque lo que está bastante correcto y no se necesitaría ninguna limpieza adicional.

# Objetivo

Generar un model de regresión capaz de predecir el valor del seguro en base a las características del cliente.

* Aplicar las técnicas oportunas de procesamiento de datos

* Valorar diferentes modelos de regresión

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

## Preprocesamiento

### 1. Codificación de variables

Observamos que el dataset consta de algunas variables categóricas que de cara a entrenar un algoritmo deberán ser convertidas a formato numérico.

In [7]:
# Convertir 'sex' a valores numéricos
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'male' else 0)

# Convertir 'smoker' a valores numéricos
df['smoker'] = df['smoker'].apply(lambda x: 1 if x == 'yes' else 0)

# Mapear las regiones a valores numéricos
df['region'] = df['region'].map({'northeast': 1, 'northwest': 2, 'southeast': 3, 'southwest': 4})


In [8]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,4,16884.92400
1,18,1,33.770,1,0,3,1725.55230
2,28,1,33.000,3,0,3,4449.46200
3,33,1,22.705,0,0,2,21984.47061
4,32,1,28.880,0,0,2,3866.85520


### 2. División de datos

Separaremos el conjunto de datos en variables predictoras (X) y la variable objetivo (y).

In [9]:
X = df.drop('charges', axis=1)
y = df['charges']

### 3. División entrenamiento y prueba

Dividiremos los datos en conjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Modelos de regresión

En esta práctica vamos a realizar el test de 3 algoritmos de regresión muy conocidos, Linear Regression, Random Forest y Decision Tree, y a compararlos entre ellos.

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Inicializar los modelos
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Decision Tree': DecisionTreeRegressor()
}

# Entrenar y evaluar los modelos
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Modelo: {name}")
    print(f"MSE: {mse}")
    print(f"R^2: {r2}")
    print()


Modelo: Linear Regression
MSE: 33635210.4311784
R^2: 0.7833463107364539

Modelo: Random Forest
MSE: 21456007.0452239
R^2: 0.8617959268391134

Modelo: Decision Tree
MSE: 42541672.401831545
R^2: 0.7259773268802148



De estos resultados llegamos a la siguiente conclusión:


*   El Random Forest muestra el MSE más bajo, lo que indica que tiene la menor cantidad de error en las predicciones.

*   El Random Forest también muestra el coeficiente de determinación más alto (R^2), lo que significa que explica la mayor parte de la variabilidad en los datos.

*   La Regresión Lineal tiene un rendimiento intermedio en términos de MSE y R^2.
El Árbol de Decisión tiene el MSE más alto y el R^2 más bajo, lo que sugiere que no se ajusta tan bien a los datos como los otros modelos.

*   Basado en las métricas de evaluación, el modelo de Random Forest parece ser el mejor para este conjunto de datos, seguido por la Regresión Lineal.
El Árbol de Decisión parece ser el menos adecuado para este conjunto de datos debido a su mayor MSE y menor R^2.




## Ensemble

Crearemos un modelo de ensamblado utilizando los modelos con mejor rendimiento.

In [13]:
from sklearn.ensemble import VotingRegressor

# Crear modelo de ensamblado
ensemble_model = VotingRegressor(estimators=list(models.items()))
ensemble_model.fit(X_train, y_train)

# Evaluación del modelo de ensamblado
y_pred_ensemble = ensemble_model.predict(X_test)
mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
r2_ensemble = r2_score(y_test, y_pred_ensemble)
print("Modelo de Ensamblado:")
print(f"MSE: {mse_ensemble}")
print(f"R^2: {r2_ensemble}")
print()


Modelo de Ensamblado:
MSE: 23229667.424223404
R^2: 0.8503712899872916



En general, el modelo de ensamblado ofrece un rendimiento sólido en términos de MSE y R^2. Esto sugiere que la combinación de diferentes modelos puede ayudar a mejorar la capacidad predictiva en comparación con los modelos individuales. En conclusión, el modelo de ensamblado parece ser una buena opción para este conjunto de datos, brindando un equilibrio entre precisión y capacidad explicativa.

# Conclusiones

Como conclusión final:



1.   Basado en las métricas de evaluación y la importancia de las características, recomendaríamos el uso del modelo de Random Forest o el modelo de ensamblado para predecir los costos del seguro médico en base a las características del cliente.




In [14]:
# Ver características más importantes en Bosques Aleatorios
if 'Random Forest' in models:
    rf_model = models['Random Forest']
    feature_importances_rf = rf_model.feature_importances_
    feature_importance_dict_rf = dict(zip(X.columns, feature_importances_rf))
    sorted_features_rf = sorted(feature_importance_dict_rf.items(), key=lambda x: x[1], reverse=True)
    print("Características más importantes en Bosques Aleatorios:")
    for feature, importance in sorted_features_rf:
        print(f"{feature}: {importance}")

# Ver características más importantes en Árbol de Decisión
if 'Decision Tree' in models:
    dt_model = models['Decision Tree']
    feature_importances_dt = dt_model.feature_importances_
    feature_importance_dict_dt = dict(zip(X.columns, feature_importances_dt))
    sorted_features_dt = sorted(feature_importance_dict_dt.items(), key=lambda x: x[1], reverse=True)
    print("\nCaracterísticas más importantes en Árbol de Decisión:")
    for feature, importance in sorted_features_dt:
        print(f"{feature}: {importance}")


Características más importantes en Bosques Aleatorios:
smoker: 0.611320608667999
bmi: 0.21182834585466484
age: 0.13537139553472288
children: 0.02042836761572025
region: 0.014446392949423023
sex: 0.006604889377470086

Características más importantes en Árbol de Decisión:
smoker: 0.6095289619759388
bmi: 0.20734956658295114
age: 0.1334671579646144
region: 0.023525987120712973
children: 0.019514401499112922
sex: 0.006613924856669782


2.   La importancia de las características reveló que el índice de masa corporal (BMI) y el hábito de fumar (smoker) son las características más influyentes en la predicción del costo del seguro médico.